# ON Model

Remember to first run the example simulation before an execution of this notebook =)

In [ ]:
import numpy as np
import pandas as pd

from pystatplottools.pdf_env.loading_figure_mode import loading_figure_mode
fma, plt = loading_figure_mode(develop=True) # develop=False will export the generated figures as pngs into "./data/RectangleData"
plt.style.use('seaborn-dark-palette')

if 'root_dir' not in locals():
    # Navigate to project root directory
    import os
    os.chdir("../")
    root_dir = os.getcwd()

# To be able to compute custom measures
import sys
sys.path.append("./../../python_scripts")
    
mcmc_model_dir = "ONModelMetropolis/"
mcmc_data_dir = root_dir + "/data/" + mcmc_model_dir
mcmc_results_dir = root_dir + "/results/" + mcmc_model_dir

data_dir = root_dir + "/data/" + mcmc_model_dir
results_dir = root_dir + "/results/" + mcmc_model_dir

## MCMC Results

### Expectation Values

In [ ]:
from mcmctools.modes.expectation_value import load_expectation_value_results
expectation_values = load_expectation_value_results(files_dir="ONModelMetropolis")
# Insert Kappa as column (as floating number)
expectation_values.insert(0, "Kappa", expectation_values.index.values.astype(np.float))
expectation_values = expectation_values.sort_values("Kappa")
expectation_values

In [ ]:
# Computation of the two point correlator
total_mean = expectation_values.loc[:, ("ExpVal", "Mean", slice(None))].values.mean(axis=1)
two_point_correlator = expectation_values["ExpVal", "TwoPointCorrelation", ""].values - total_mean * total_mean

In [ ]:
fig, axes = fma.newfig(1.4, ncols=3, figsize=(12, 4))

axes[0].plot(expectation_values["Kappa"], expectation_values["ExpVal", "SecondMoment", ""], "o-")
axes[0].set_xlabel("Kappa")
axes[0].set_ylabel("SecondMoment")

axes[1].plot(expectation_values["Kappa"], expectation_values["ExpVal", "Energy", ""], "o-")
axes[1].set_xlabel("Kappa")
axes[1].set_ylabel("Energy")

axes[2].plot(expectation_values["Kappa"], two_point_correlator, "o-")
axes[2].set_xlabel("Kappa")
axes[2].set_ylabel("Two Point Correlator")


plt.tight_layout()

fma.savefig(results_dir, "expectation_values")

## Configurations as Pytorch Dataset

We show how the mcmc configurations can be stored and loaded as a .pt file.

(See also python_scripts/loading_configurations.py and python_scripts/pytorch_data_generation.py)

### Preparation

In [ ]:
data_generator_args = {
    # ConfigDataGenerator Args
    "data_type": "target_param", # Parameters as target. In contrast, target_config has configs as target
    # Use energy and two point correlation as target - since the two_point_correlation is a post_measure
    # it needs to be computed when the data is loaded. This is done by defining transform=True
    "labels": [["Kappa", "", ""], ["Energy", "", ""], ["TwoPointCorrelation", "", ""]], # (default is the running_parameter)  
    # Args for ConfigurationLoader
    "path": mcmc_data_dir,
    "total_number_of_data_per_file": 10000,
    "identifier": "expectation_value",
    "running_parameter": "kappa",
    "transform": True, # Indicates that there exists a raw_transformer.py file with a transformer function in
    # the "path" + raw/ directory. The path of the raw_transformer.py file can be adpated with the parameter "transformer_path"
    "chunksize": 100  # If no chunksize is given, all data is loaded at once
}

# Prepare in memory dataset
from pystatplottools.pytorch_data_generation.data_generation.datagenerationroutines import prepare_in_memory_dataset
from mcmctools.pytorch.data_generation.datagenerationroutines import data_generator_factory

prepare_in_memory_dataset(
    root=data_dir,
    batch_size=89,
    data_generator_args=data_generator_args,
    data_generator_name="BatchConfigDataGenerator",
    data_generator_factory=data_generator_factory
)

### Generating and Loading the Dataset

In [ ]:
# Load in memory dataset
from pystatplottools.pytorch_data_generation.data_generation.datagenerationroutines import load_in_memory_dataset

# The dataset is generated and stored as a .pt file in the data_dir/data directory the first time this function is called. Otherwise the .pt is loaded.
data_loader = load_in_memory_dataset(
    root=data_dir, batch_size=128, data_generator_factory=data_generator_factory, slices=None, shuffle=True,
    num_workers=0, rebuild=False
    # sample_data_generator_name="ConfigDataGenerator"  # optional: for a generation of new samples
)

# Load training data
for batch_idx, batch in enumerate(data_loader):
    data, target = batch
    # print(batch_idx, len(data))

In [ ]:
("Kappa", target[0][0].item()), ("Energy", target[0][1].item()), ("TwoPointCorrelation", target[0][2].item())

### Inspection of the Dataset - Sample Visualization

In [ ]:
from pystatplottools.visualization import sample_visualization

config_dim = (8, 8)  # Dimension of the data
num_std=1

# Random samples
config, label = data_loader.dataset.get_random_sample()
batch, batch_label = data_loader.dataset.get_random_batch(108)

In [ ]:
# Single Sample
sample_visualization.fd_im_single_sample(sample=config, label=label, config_dim=config_dim, num_std=num_std,
                                         fma=fma, filename="single_sample", directory=results_dir, figsize=(10, 4));

In [ ]:
# Batch with labels
sample_visualization.fd_im_batch(batch, batch_labels=batch_label, num_samples=25, dim=(5, 5),
                                 config_dim=config_dim, num_std=num_std,
                                 fma=fma, filename="batch", directory=results_dir, width=2.3, ratio=1.0, figsize=(12, 12));

In [ ]:
# Batch grid
sample_visualization.fd_im_batch_grid(batch, config_dim=config_dim, num_std=num_std,
                                      fma=fma, filename="batch_grid", directory=results_dir);

### Loading the Data without Pytorch

In [ ]:
# Load all data
from mcmctools.loading.loading import load_data

# skipcols=[] Can be used to load only certain columns of the different files
data, filenames = load_data(files_dir=mcmc_model_dir, running_parameter="kappa",
                            identifier="expectation_value")  # , skipcols=["Config"])
from mcmctools.utils.json import load_configs
sim_params, execution_params, running_parameter = load_configs(
    files_dir="ONModelMetropolis", mode="expectation_value", sim_base_dir="./")

data